# Equal-Weight S&P 500 Index Fund

## Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this project is to create a Python script that will accept the value of the portfolio and show how many shares of each S&P 500 constituent one should purchase to get an equal-weight version of the index fund.

## Library Imports

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Importing List of Stocks

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring API Token

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making API Call

Structure our API calls to IEX cloud. 

Need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'SAGEN AACLTGABO SDK LM(S)NREL/TQE',
 'calculationPrice': 'close',
 'open': 142.49,
 'openTime': 1655243611883,
 'openSource': 'flocfiia',
 'close': 138.29,
 'closeTime': 1641487168815,
 'closeSource': 'fiilcofa',
 'high': 137.31,
 'highTime': 1664976937105,
 'highSource': 'rmeairetI  p ilcEXe',
 'low': 140.77,
 'lowTime': 1680560543777,
 'lowSource': 'ea5ceypi eiuendlrdmt 1 ',
 'latestPrice': 137.22,
 'latestSource': 'Close',
 'latestTime': 'February 8, 2021',
 'latestUpdate': 1673824447772,
 'latestVolume': 73411307,
 'iexRealtimePrice': 137.43,
 'iexRealtimeSize': 21,
 'iexLastUpdated': 1690056950502,
 'delayedPrice': 139.3,
 'delayedPriceTime': 1668977364673,
 'oddLotDelayedPrice': 141.472,
 'oddLotDelayedPriceTime': 1685204927641,
 'extendedPrice': 140.6,
 'extendedChange': -0.32,
 'extendedChangePercent': -0.00229,
 'extendedPriceTime': 1651058892081,
 'previousClose': 140.63,
 'previousVolume': 77336232,
 'chang

## Parsing API Call


In [5]:
data['latestPrice']
data['marketCap']

2384101831031

## Adding Stocks Data to a Pandas DataFrame


In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,137.22,2384101831031,N/A


## Looping Through The Tickers in the List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)


In [9]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,128.49,39308528874,N/A
1,AAL,18.37,10968675348,N/A
2,AAP,165.99,11027048993,N/A
3,AAPL,139.17,2332000501074,N/A
4,ABBV,109.57,190989547386,N/A
...,...,...,...,...
500,YUM,104.89,31799562557,N/A
501,ZBH,167.25,34708224223,N/A
502,ZBRA,431.60,22531545904,N/A
503,ZION,48.71,8075705991,N/A


## Using Batch API Calls to Improve Performance


IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5.

Split list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,127.45,39422615802,N/A
1,AAL,18.24,11090169639,N/A
2,AAP,159.80,10754412999,N/A
3,AAPL,143.39,2383430686495,N/A
4,ABBV,111.12,194143051968,N/A
...,...,...,...,...
500,YUM,108.18,32081826642,N/A
501,ZBH,165.19,34407792521,N/A
502,ZBRA,434.31,23314706157,N/A
503,ZION,48.79,8335436337,N/A


## Calculating the Number of Shares to Buy


In [12]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100


In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,127.45,39422615802,0
1,AAL,18.24,11090169639,0
2,AAP,159.80,10754412999,0
3,AAPL,143.39,2383430686495,0
4,ABBV,111.12,194143051968,0
...,...,...,...,...
500,YUM,108.18,32081826642,0
501,ZBH,165.19,34407792521,0
502,ZBRA,434.31,23314706157,0
503,ZION,48.79,8335436337,0


## Formatting Excel Output

### Initializing XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats Needed For `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`.
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of `.xlsx` File

Use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

An example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [16]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


In [17]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Excel Output


In [18]:
writer.save()